In [80]:
import numpy as np
import numpy.typing as npt

# CM-DVR functions
# generate grids - https://github.com/ScottHabershon/PS/blob/main/src/grids.py

def get_exact_grid(x_min: float, x_max: float, exact_grid_size: int) -> npt.NDArray:
    """
    Generates linearly space grid to be used in the exact solver.

    :param x_min: minimum value of the grid
    :type x_min: float
    :param x_max: maximium value of the grid
    :type x_max: float
    :param exact_grid_size: size of the grid used in the exact solver
    :type exact_grid_size: int
    :return: xgrid_exact
    :rtype: npt.NDArray
    """
    xgrid_exact = np.linspace(x_min, x_max, exact_grid_size)
    dx = xgrid_exact[1] - xgrid_exact[0]
    return xgrid_exact, dx

# generate 1D potentials
# 0.5kx^2 - harmonic oscillator

def harmonic_oscillator(k: float, grid: npt.NDArray):
    """
    Sets up a harmonic oscillator potential on a provided grid.

    Args:
        k: force/spring constant

    Returns:
        v[:]: 1-D potential energy, calculated at grid points.
    """
    v = 0.5 * k * (grid ** 2)
    return v

# solve Schrödinger eqn for potentials - DVR
# CM DVR taken from https://github.com/ScottHabershon/PS/blob/main/src/exact_solver.py

def colbert_miller_DVR(ngrid, x, m, v):
    """
    Performs Colbert-Miller DVR solution of 1-D Schrodinger equation.

    Args:
        ngrid: Number of grid points
        x[:]: Positions of grid points.
        m: mass of particle
        v[:]: 1-D potential energy, calculated at grid points.

    Returns:
        c[:,0]: The ground-state wavefunction.
    """

    #  Atomic units:
    hbar = 1.0

    # Set grid spacing.
    dx = x[1] - x[0]

    #  Set up potential energy matrix.
    V = np.zeros((ngrid, ngrid))
    for i in range(ngrid):
        V[i, i] = v[i]

    #  Set up kinetic energy matrix.
    T = np.zeros((ngrid, ngrid))
    for i in range(ngrid):
        for j in range(ngrid):
            if i == j:
                T[i, j] = ((hbar ** 2) * np.pi ** 2) / (6 * m * dx ** 2)
            else:
                T[i, j] = ((hbar ** 2) * (-1.0) ** (i - j)) / (m * dx ** 2 * (i - j) ** 2)

    # Create the Hamiltonian matrix:
    H = T + V

    #  Solve the eigenvalue problem using the linalg.eigh
    E, c = np.linalg.eigh(H)
    # E = E.astype('float128',copy=False) # comment out for mac systems

    #  Normalize each eigenfunction using simple quadrature.
    for i in range(ngrid):
        csum = np.trapz(np.conj(c[:,i]) * c[:,i], x)
        c[:, i] = c[:, i] / np.sqrt(csum)
        E[i] = np.real(E[i])

    return c, E, H

# calculate Kubo TCF from CM-DVR results
# TODO - FIX - this is incorrect, it should give different values for different times
def Kubo_TCF(beta, grid, E, c, dx, times):
    for i in range(0, len(E)):
        Z = 1 / np.exp(-beta * E[i])
        for j in range(0, len(E)):
            C_1 = np.exp(-1j * (E[i]- E[j]) * times) # hbar = 1 - assume atomic units
            Aij = np.trapz(np.conj(c[:,i]) * grid[i] * c[:,j], dx = dx)
            Bji = np.trapz(np.conj(c[:,j]) * grid[j] * c[:,i], dx = dx)
            if i != j: # i == j -> 1-exp(0) = 0, AND div by 0 occurs
                C_2 = (1 - np.exp(-beta * (E[j]-E[i]))) / (E[j]-E[i])
        C = (1 / beta*Z) * (1/Z) * C_1 * Aij * Bji * C_2
        C = C.real
    return C

In [81]:
# initialise variables
k = 1
m = 1

initial_energy = 2
initial_position = 0.1
initial_velocity = 1

dt = 0.1
max_time = 10

# run CM DVR
x_min = -5
x_max = 5
grid_size = 11

grid, dx = get_exact_grid(x_min,x_max,grid_size)
v = harmonic_oscillator(k,grid)

# returns c - ground state wavefunction - ith column of c contains wavefunction phi(i), E - eigenvalues, H - hamiltonian, of the system
c, E, H = colbert_miller_DVR(grid_size, grid, m, v)

t = np.arange(0,max_time, dt) # same times used in classical MD
beta = 1

In [82]:
C = Kubo_TCF(beta, grid, E, c, dx, t)
C

array([25.52081878, 25.52081878, 25.52081878, 25.52081878, 25.52081878,
       25.52081878, 25.52081878, 25.52081878, 25.52081878, 25.52081878,
       25.52081878, 25.52081878, 25.52081878, 25.52081878, 25.52081878,
       25.52081878, 25.52081878, 25.52081878, 25.52081878, 25.52081878,
       25.52081878, 25.52081878, 25.52081878, 25.52081878, 25.52081878,
       25.52081878, 25.52081878, 25.52081878, 25.52081878, 25.52081878,
       25.52081878, 25.52081878, 25.52081878, 25.52081878, 25.52081878,
       25.52081878, 25.52081878, 25.52081878, 25.52081878, 25.52081878,
       25.52081878, 25.52081878, 25.52081878, 25.52081878, 25.52081878,
       25.52081878, 25.52081878, 25.52081878, 25.52081878, 25.52081878,
       25.52081878, 25.52081878, 25.52081878, 25.52081878, 25.52081878,
       25.52081878, 25.52081878, 25.52081878, 25.52081878, 25.52081878,
       25.52081878, 25.52081878, 25.52081878, 25.52081878, 25.52081878,
       25.52081878, 25.52081878, 25.52081878, 25.52081878, 25.52